In [1]:
import pandas as pd
import soynlp
from soynlp.noun import LRNounExtractor_v2
from soynlp import DoublespaceLineCorpus
from soynlp.word import WordExtractor
from soynlp.tokenizer import LTokenizer

import warnings
warnings.filterwarnings('ignore')

In [6]:
def clean_sents_df(company):
    "C:\sh\study\krx데이콘\krx_2022\junho\src\2year_time\삼성전자_2year.csv"
    try:
        target = pd.read_csv(f'C:/sh/study/krx데이콘/krx_2022/junho/src/2year_time/{company}_2year.csv',encoding='utf8').drop('Unnamed: 0',axis=1)
    except:
        target = pd.read_csv(f'C:/sh/study/krx데이콘/krx_2022/junho/src/2year_time/{company}_2year.csv',encoding='utf8')

    if company == 'sampro':
        target.rename(columns={'comment':'댓글'},inplace=True)
    df = target
    df['정제된 댓글'] = df['댓글'].str.replace('\\[삭제된 게시물의 답글\\]',' ')
    df['정제된 댓글'] = df['정제된 댓글'].str.replace('답글:',' ')
    df['정제된 댓글'] = df['정제된 댓글'].str.replace('[^가-힣]',' ').str.replace(' +',' ').str.strip()
    df = df[df['정제된 댓글'] != '']
    df = df.reset_index(drop=True)
    return  df

def corpus_save(company):
    df = clean_sents_df(company)
    df['정제된 댓글 길이'] = [len(str(i)) for i in df['정제된 댓글']]
    df = df[df['정제된 댓글 길이'] > 5]

    tp = [str(i) for i in list(df['정제된 댓글'])]
    save = '\n'.join(tp)
    f = open(r"C:\sh\study\krx데이콘\krx_2022\junho\corpus_target.txt", 'a',encoding='utf8')
    f.write(save)
    f.close()

    
def corpus_init():
    ktop30_company = pd.read_excel(r'C:\sh\study\krx데이콘\krx_2022\junho\KODEX_KTOP_30_20220629.xlsx',header=2).drop(0,axis=0)['종목명']
    company_set = list(ktop30_company)
    company_set.append('sampro')
    f = open(r"C:\sh\study\krx데이콘\krx_2022\junho\corpus_target.txt", 'w',encoding='utf8')
    f.write('')
    f.close()
    for company in company_set:
        corpus_save(company)

def return_tokenizer():
    corpus = DoublespaceLineCorpus(r"C:\sh\study\krx데이콘\krx_2022\junho\corpus_target.txt",iter_sent=True)
    noun_extractor = LRNounExtractor_v2(verbose=True)
    nouns = noun_extractor.train_extract(corpus)
    scores = {word:score.score for word, score in nouns.items()}
    tokenizer = LTokenizer(scores=scores)
    return tokenizer

In [7]:
corpus_init()
tokenizer = return_tokenizer()

[Noun Extractor] use default predictors
[Noun Extractor] num features: pos=3929, neg=2321, common=107
[Noun Extractor] counting eojeols
[EojeolCounter] n eojeol = 1249734 from 2753662 sents. mem=0.419 Gb                    
[Noun Extractor] complete eojeol counter -> lr graph
[Noun Extractor] has been trained. #eojeols=9746776, mem=2.921 Gb
[Noun Extractor] batch prediction was completed for 289636 words
[Noun Extractor] checked compounds. discovered 280851 compounds
[Noun Extractor] postprocessing detaching_features : 249805 -> 199708
[Noun Extractor] postprocessing ignore_features : 199708 -> 199316
[Noun Extractor] postprocessing ignore_NJ : 199316 -> 196316
[Noun Extractor] 196316 nouns (280851 compounds) with min frequency=1
[Noun Extractor] flushing was done. mem=3.326 Gb                    
[Noun Extractor] 71.68 % eojeols are covered


In [8]:
target_df = clean_sents_df('삼성전자')
target_df['토큰화 댓글'] = [tokenizer(str(i)) for i in target_df['정제된 댓글']]
target_df

,날짜,댓글,조회수,좋아요,싫어요,정제된 댓글,토큰화 댓글
0,2022-07-11 23:42,요즘 한전 게시판 조용하네!ㅋ,2,0,0,요즘 한전 게시판 조용하네,"[요즘, 한전, 게시판, 조용, 하네]"
1,2022-07-11 23:42,1% 올려야 함,4,0,0,올려야 함,"[올려야, 함]"
2,2022-07-11 23:42,=자신들잘못된죄들=모르는=내로남불들=척결...,3,0,0,자신들잘못된죄들 모르는 내로남불들 척결,"[자신들, 잘못된죄들, 모르는, 내로남불, 들, 척결]"
3,2022-07-11 23:42,난 내일 가요,9,0,0,난 내일 가요,"[난, 내일, 가요]"
4,2022-07-11 23:42,실적으로 이만한 실적내는 기업 있으면 나...,6,0,0,실적으로 이만한 실적내는 기업 있으면 나,"[실적, 으로, 이만, 한, 실적, 내는, 기업, 있으면, 나]"
...,...,...,...,...,...,...,...
952447,2020-06-01 00:09,내8자,707,1,3,내 자,"[내, 자]"
952448,2020-06-01 00:08,하락낙폭저지 의 길,682,2,0,하락낙폭저지 의 길,"[하락낙폭저지, 의, 길]"
952449,2020-06-01 00:07,주주님들 오랜만에 오르니까 좋네요 ㅎㅎㅎ,697,1,0,주주님들 오랜만에 오르니까 좋네요,"[주주님, 들, 오랜만, 에, 오르니까, 좋네요]"
952450,2020-06-01 00:03,코덱스200선물인버스 곱배기..,725,0,0,코덱스 선물인버스 곱배기,"[코덱스, 선물, 인버스, 곱배기]"


# 레이블링 & 랜덤추출

---

In [14]:
def labeling(target_df):
    
    f = open(r"C:\sh\study\krx데이콘\krx_2022\junho\src\fear_words.txt", 'r',encoding='utf8')
    words = f.readlines()
    f.close()
    fear_words_set = {word.strip('\n') for word in words}

    f = open(r"C:\sh\study\krx데이콘\krx_2022\junho\src\greed_words.txt", 'r',encoding='utf8')
    words = f.readlines()
    f.close()
    greed_words_set = {word.strip('\n') for word in words}
    
    label_score = []
    for token_list in target_df['토큰화 댓글']:
        sent_score = 0
        for token in token_list:
            if token in fear_words_set:
                sent_score -= 1
            elif token in greed_words_set:
                sent_score += 1

        if sent_score < 0:
            label_score.append(-1)
        elif sent_score > 0:
            label_score.append(1)
        else:
            label_score.append(0)
            
    target_df['label'] = label_score
    
    return target_df


In [12]:
def setting_train_data():
    company = list(pd.read_excel(r'C:\sh\study\krx데이콘\krx_2022\junho\KODEX_KTOP_30_20220629.xlsx',header=2)['종목명'][1:])
    dominance = list(pd.read_excel(r'C:\sh\study\krx데이콘\krx_2022\junho\KODEX_KTOP_30_20220629.xlsx',header=2)['비중(%)'][1:])

    train_data = pd.DataFrame(columns=['날짜','댓글','조회수','좋아요','싫어요','정제된 댓글','토큰화 댓글','label'])
    for idx in range(30):
        target_df = clean_sents_df(company[idx])
        target_df['토큰화 댓글'] = [tokenizer(str(i)) for i in target_df['정제된 댓글']]
        
        label_df = labeling(target_df)
        
        label_df = label_df[label_df['label'] != 0]
        label_df = label_df.sample(int(30000*dominance[idx])+1,replace=True) # replace option : 샘플수가 부족할경우 중복이 생김 ** 추후 수정
        train_data = train_data.append(label_df)
    
    return train_data

In [15]:
train_data = setting_train_data()
train_data.reset_index(inplace=True)
train_data.drop('index',axis=1,inplace=True)
train_data

,날짜,댓글,조회수,좋아요,싫어요,정제된 댓글,토큰화 댓글,label
0,2021-08-12 10:50,개미가 털고 나가야 오른다,204,1,0,개미가 털고 나가야 오른다,"[개미가, 털고, 나가야, 오른다]",1
1,2021-01-19 10:52,공포에 사고 환희에 팔아라,262,1,0,공포에 사고 환희에 팔아라,"[공포, 에, 사고, 환희, 에, 팔아라]",-1
2,2021-10-21 10:56,8만원을 위하여~~~~다음주 실적 발표하...,435,0,2,만원을 위하여 다음주 실적 발표하,"[만원, 을, 위하여, 다음주, 실적, 발표, 하]",-1
3,2021-08-02 17:05,역시 개미가 나가야 오른다,299,1,1,역시 개미가 나가야 오른다,"[역시, 개미가, 나가야, 오른다]",1
4,2021-04-06 14:47,낼 실적발표,364,1,0,낼 실적발표,"[낼, 실적, 발표]",-1
...,...,...,...,...,...,...,...,...
30010,2020-08-05 14:41,실적발표 언제 합니까.,303,1,0,실적발표 언제 합니까,"[실적, 발표, 언제, 합니까]",-1
30011,2020-08-05 17:42,실적발표?,1098,0,1,실적발표,"[실적, 발표]",-1
30012,2021-01-22 12:50,지수는 하늘을 나는데 증권주는 안 오른다...,316,4,3,지수는 하늘을 나는데 증권주는 안 오른다,"[지수, 는, 하늘, 을, 나는데, 증권주, 는, 안, 오른다]",1
30013,2020-12-02 11:29,ㅋㅋㅋ 어제 정말 오랫만에 쫌 오른다 싶...,249,3,1,어제 정말 오랫만에 쫌 오른다 싶,"[어제, 정말, 오랫만, 에, 쫌, 오른다, 싶]",1


---

In [16]:
# 삼프로
target_df = clean_sents_df('sampro')
target_df['토큰화 댓글'] = [tokenizer(str(i)) for i in target_df['정제된 댓글']]
label_df = labeling(target_df)

label_df = label_df[label_df['label'] != 0]
label_df = label_df.sample(10000,replace=True) # replace option : 샘플수가 부족할경우 중복이 생김 ** 추후 수정
train_data = train_data.append(label_df)

In [17]:
train_data = train_data.loc[:,['정제된 댓글','label']].reset_index().drop('index',axis=1)

In [18]:
train_data

,정제된 댓글,label
0,개미가 털고 나가야 오른다,1
1,공포에 사고 환희에 팔아라,-1
2,만원을 위하여 다음주 실적 발표하,-1
3,역시 개미가 나가야 오른다,1
4,낼 실적발표,-1
...,...,...
40010,엄청난 열기와 성원에 힘입어 진행된 하반기 투자전략 세미나 지금 온라인으로 한 달간...,1
40011,강의 신청하기 투자의 정답을 찾는 책읽기 시즌 사경인의 실전투자 아카데미 리마스터 ...,1
40012,자난 월은 월을 이겨낸 분들의 날이였죠 지금도 공포에 매수 한 분들의 좋은 결과 있...,-1
40013,에 개인투자자들이 너무 많이 몰려서 손실을 볼까 염려하는 마음에 걱정해주시는 것은 ...,1


In [22]:
for i in range(len(train_data)):
    if train_data["label"][i] == -1:
        train_data["label"][i] = 0


In [23]:
train_data

,정제된 댓글,label
0,개미가 털고 나가야 오른다,1
1,공포에 사고 환희에 팔아라,0
2,만원을 위하여 다음주 실적 발표하,0
3,역시 개미가 나가야 오른다,1
4,낼 실적발표,0
...,...,...
40010,엄청난 열기와 성원에 힘입어 진행된 하반기 투자전략 세미나 지금 온라인으로 한 달간...,1
40011,강의 신청하기 투자의 정답을 찾는 책읽기 시즌 사경인의 실전투자 아카데미 리마스터 ...,1
40012,자난 월은 월을 이겨낸 분들의 날이였죠 지금도 공포에 매수 한 분들의 좋은 결과 있...,0
40013,에 개인투자자들이 너무 많이 몰려서 손실을 볼까 염려하는 마음에 걱정해주시는 것은 ...,1


In [27]:
type(train_data)

pandas.core.frame.DataFrame

In [29]:
path = "C:/sh/study/krx데이콘/krx_2022/sh"

In [30]:
train_data.to_csv(path + "/train_data.csv")

In [32]:
df = pd.read_csv("train_data.csv")

In [33]:
df_nn = df[["정제된 댓글","label"]]

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(df_nn["정제된 댓글"],df_nn["label"],test_size=0.2,random_state=11)

In [44]:
df_train = pd.concat([x_train,y_train],axis = 1)
df_test = pd.concat([x_test,y_test],axis = 1)

In [45]:
df_train

,정제된 댓글,label
21613,분기 실적만 기다리며 추매중,0
26881,코로나 수혜주 아니였냐 실적도 좋은데,0
25626,실적발표를 빨리한다는건좋은신호로 받아들였,0
5718,허매수가 매도의 배다 절대못오른다,1
21414,누루기 분기 실적,0
...,...,...
32081,만 구독 감사 이벤트는 본 영상에 댓글을 달아도 참여하실 수 있습니다 경품응모 신청...,1
7259,월 실적발표 전까지는,0
21584,이제 오른다,1
36543,요약 노 트 손재권 대표님 더밀크 시간이 없거나 이해가 힘드신분들을 위한 댓글입니다...,0


In [46]:
df_test

,정제된 댓글,label
38107,유럽에서 수소에 관심이 있는 이유에 대해 제대로 설명이 안되는 것 같아 추가하자면 ...,0
37211,강의 신청하기 투자의 정답을 찾는 책읽기 시즌 사경인의 실전투자 아카데미 리마스터 ...,1
18197,공포의 기 온다 삼성이 본사 사옥까,0
32795,만 구독 감사 이벤트는 본 영상에 댓글을 달아도 참여하실 수 있습니다 경품응모 신청...,1
27323,공포에사라,0
...,...,...
39336,삼프로 가 준비한 다양한 프리미엄 콘텐츠 삼프로 주식대학 업종 기초 교육과 실전 투...,1
28496,기아차 실적,0
13105,동학동지들이여 공포에사라,0
17157,실적에 비해 너무저평가 된 주식,0
